In [1]:
import os
import pandas as pd
from PIL import Image
from transformers import AutoImageProcessor, AutoModelForImageClassification, pipeline
from tqdm.auto import notebook_tqdm
import torch

In [2]:
model_name = "microsoft/swinv2-large-patch4-window12to24-192to384-22kto1k-ft"
processor = AutoImageProcessor.from_pretrained(model_name, return_tensor="pt")

preprocessor_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

g:\내 드라이브\DACON\DACON-Bird_Image_Classification\bird\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hero9\.cache\huggingface\hub\models--microsoft--swinv2-large-patch4-window12to24-192to384-22kto1k-ft. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Could not find image processor class in the image

In [3]:
model = AutoModelForImageClassification.from_pretrained("./results/swinv2_384/checkpoint-1187/")
model.eval()

classifier = pipeline("image-classification", model=model, image_processor=processor)

In [4]:
test_df = pd.read_csv("./datas/test.csv")
test_df.head()

,id,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg


In [5]:
test_file_name = test_df['img_path'].to_list()

predict_labels = []

for idx, name in notebook_tqdm(enumerate(test_file_name), total=len(test_file_name)):
    image = Image.open(os.path.join("./datas/", name.strip('./')))
    predict_labels.append(classifier(image)[0]['label'])

  0%|          | 0/6786 [00:00<?, ?it/s]

In [ ]:
submission_df = pd.read_csv("./datas/sample_submission.csv")
submission_df.head()
submission_df['label'] = predict_labels
submission_df.to_csv("./results/swinv2_384/submission_transformers_ckp1184.csv", index=False)